# Topic disaggregation

### Repeat the same process on data previously disaggregated according to the categories of the vote

In [6]:
import pandas as pd
import networkx as nx
import numpy as np
import re

import glob, os
import itertools
from tqdm import tqdm

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
from shapely.geometry import LineString
from matplotlib.lines import Line2D

In [7]:
import sys
sys.path.insert(0, '../local/bin')

In [8]:
def create_members_df(members, party_codes):
    temp_congress = members.groupby('icpsr', as_index=False)[['congress']].agg(lambda x: list(x))                                                           # group by icpsr and aggregate the congress numbers into a list
    temp_party = members.groupby('icpsr', as_index=False)[['party_code']].agg(lambda x: list(set(x)))                                                       # group by icpsr and aggregate the party codes into a list
    temp_congress = temp_congress.merge(temp_party)                                                                                                         # merge the two dataframes
    temp_congress['bioname'] = temp_congress['icpsr'].map(members[['icpsr', 'bioname']].set_index('icpsr').to_dict()['bioname'])                            # insert the bioname based on the icpsr 
    temp_congress['state_abbrev'] = temp_congress['icpsr'].map(members[['icpsr', 'state_abbrev']].set_index('icpsr').to_dict()['state_abbrev'])             # insert the state_abbrev based on the icpsr
    party_codes_dic = party_codes[['party_name', 'party_code']].set_index('party_code').to_dict()['party_name']                                             # create a dictionary for the party codes
    temp_congress['party_name'] = temp_congress['party_code'].apply(lambda x: [party_codes_dic[y] for y in x])                                              # insert the party name based on the party code
    return temp_congress

In [9]:
members = pd.read_csv('../dataset/HSall_members.csv')
party_codes = pd.read_csv('../dataset/HSall_parties.csv')

members_info = create_members_df(members, party_codes)

member_party_dict = members_info.set_index('icpsr')['party_name'].to_dict()		# member_id -> party_name

In [10]:
def create_edgelist_from_congress(congress, members_party_dict):
	edgelist = pd.DataFrame()

	for voteid in tqdm(set(congress['id'])):                 # iterate over all votes id (ids are unique for each vote)

		temp = congress[congress['id'] == voteid]            # select the rows where the vote id is equal to the current vote id            

		yy = temp[temp['vote']=='Yea']['icpsr']              # select the icpsr of the members that voted "Yea"
		nn = temp[temp['vote']=='Nay']['icpsr']                         

		y = itertools.combinations(yy, 2)                    # all possible combinations of 2 members that voted "Yea"
		n = itertools.combinations(nn, 2)                
		o = itertools.product(yy, nn)                        # cartesian product of the 2 series

		y = pd.DataFrame(y, columns = ['source', 'target'])  # create a dataframe from the combinations of "Yea" voters
		y['weight'] = 1                                      # add a column with the weight of the edge
		y['count'] = 1                                         

		n = pd.DataFrame(n, columns = ['source', 'target'])     
		n['weight'] = 1                                         
		n['count'] = 1                                          

		o = pd.DataFrame(o, columns = ['source', 'target'])     
		o['weight'] = -1                                     # same but the link is negative                    
		o['count'] = 1                                          

		edgelist = pd.concat([edgelist, y, n, o])
		edgelist = edgelist.groupby(['source', 'target', 'weight']).sum().reset_index()     # group by source, target and weight and sum the count                  

	edgelist = pd.concat([edgelist, pd.DataFrame({
		'source': edgelist['target'],                        # new columns based on old columns: 
		'target': edgelist['source'],                        #   'newcolumn': dataframe['oldcolumn']
		'weight': edgelist['weight'],
		'count': edgelist['count']})])

	edgelist = edgelist.loc[edgelist['source'] < edgelist['target']]                    # remove duplicates
	edgelist = edgelist.groupby(['source', 'target', 'weight']).sum().reset_index()     # group by source, target and weight and sum the count
	edgelist['party'] = edgelist.apply(lambda row: 'in' if members_party_dict[row['source']] == members_party_dict[row['target']] else 'out', axis=1)   # create a column with the party of the edge

	map_votes = edgelist.groupby(['source', 'target'])['count'].sum().to_dict()                                                                         # create a dictionary with the number of votes togheter for each pair of nodes                               

	edgelist['votes_togheter'] = edgelist[['source', 'target']].apply(lambda x: map_votes[(x['source'], x['target'])], axis=1)
	edgelist['perc'] = edgelist['count']/edgelist['votes_togheter']

	return edgelist

In [11]:
congress_path = '../dataset/download_votes_merged/*'
congress_path_senate = '../dataset/download_votes_merged_senate/*'

In [12]:
# Initialize a weighted bipartite graph
G = nx.Graph()

# Iterate through each CSV file
for csv_file in tqdm(glob.glob(congress_path)):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Count the occurrences of each pair of Clausen and Peltzman categories
    counts = df.groupby(["Clausen", "Peltzman"]).size().reset_index(name="count")
    
    # Add edges to the graph with weights based on the counts
    for _, row in counts.iterrows():
        clausen_category = row["Clausen"]
        peltzman_category = row["Peltzman"]
        count = row["count"]
        
        G.add_node(clausen_category, bipartite=0)
        G.add_node(peltzman_category, bipartite=1)
        G.add_edge(clausen_category, peltzman_category, weight=count)

# Now you have a weighted bipartite graph (G) with edges representing the co-occurrences
# between Clausen and Peltzman categories and weights representing the number of occurrences.

 78%|███████▊  | 32/41 [00:21<00:07,  1.25it/s]/tmp/ipykernel_20883/3165551236.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
100%|██████████| 41/41 [00:32<00:00,  1.28it/s]


In [13]:
print('Clausen categories: ', [n for n in G.nodes() if G.nodes[n]['bipartite'] == 0])
print('Number of Clausen categories: ', len([n for n in G.nodes() if G.nodes[n]['bipartite'] == 0]))

print('Peltzman categories: ', [n for n in G.nodes() if G.nodes[n]['bipartite'] == 1])
print('Number of Peltzman categories: ', len([n for n in G.nodes() if G.nodes[n]['bipartite'] == 1]))

Clausen categories:  ['Agriculture', 'Civil Liberties', 'Foreign and Defense Policy', 'Government Management', 'Miscellaneous Policy', 'Social Welfare']
Number of Clausen categories:  6
Peltzman categories:  ['Budget Special Interest', 'Regulation General Interest', 'Regulation Special Interest', 'Domestic Social Policy', 'Defense Policy Budget', 'Defense Policy Resolutions', 'Foreign Policy Budget', 'Foreign Policy Resolutions', 'Budget General Interest', 'D. C.', 'Government Organization', 'Indian Affairs', 'Internal Organization']
Number of Peltzman categories:  13


____________________________________________________

### Disaggregate the congresses

In [34]:
clausen_cat_list = [n for n in G.nodes() if G.nodes[n]['bipartite'] == 0]
peltzman_cat_list = [n for n in G.nodes() if G.nodes[n]['bipartite'] == 1]

In [35]:
clausen_cat_dic = {x: re.sub(' ', '_', x) for x in clausen_cat_list}
peltzman_cat_dic = {x: re.sub(' ', '_', x) for x in peltzman_cat_list}

In [36]:
os.makedirs('../dataset/votes_house_clausen', exist_ok=True)
os.makedirs('../dataset/votes_senate_clausen', exist_ok=True)

os.makedirs('../dataset/votes_house_peltzman', exist_ok=True)
os.makedirs('../dataset/votes_senate_peltzman', exist_ok=True)

In [37]:
for clausen in clausen_cat_dic.values():
	os.makedirs('../dataset/votes_house_clausen/'+clausen, exist_ok=True)
	os.makedirs('../dataset/votes_senate_clausen/'+clausen, exist_ok=True)

for peltzman in peltzman_cat_dic.values():
	os.makedirs('../dataset/votes_house_peltzman/'+peltzman, exist_ok=True)
	os.makedirs('../dataset/votes_senate_peltzman/'+peltzman, exist_ok=True)

In [38]:
for csv in tqdm(glob.glob(congress_path)):
    n_congress = int(os.path.basename(csv).replace('congress_', '').replace('.csv', ''))

    df_congress = pd.read_csv(csv)

    for i in clausen_cat_dic.keys():
        filtered_df = df_congress[df_congress['Clausen'] == i]
        if not filtered_df.empty:
            filtered_df.to_csv('../dataset/votes_house_clausen/'+clausen_cat_dic[i]+'/congress_'+str(n_congress)+'.csv', index=False)

    for i in peltzman_cat_dic.keys():
        filtered_df = df_congress[df_congress['Peltzman'] == i]
        if not filtered_df.empty:
            filtered_df.to_csv('../dataset/votes_house_peltzman/'+peltzman_cat_dic[i]+'/congress_'+str(n_congress)+'.csv', index=False)

 78%|███████▊  | 32/41 [01:32<00:24,  2.75s/it]/tmp/ipykernel_20883/1185449784.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_congress = pd.read_csv(csv)
100%|██████████| 41/41 [02:16<00:00,  3.33s/it]


In [39]:
for csv in tqdm(glob.glob(congress_path_senate)):
    n_congress = int(os.path.basename(csv).replace('congress_', '').replace('.csv', ''))

    df_congress = pd.read_csv(csv)

    for i in clausen_cat_dic.keys():
        filtered_df = df_congress[df_congress['Clausen'] == i]
        if not filtered_df.empty:
            filtered_df.to_csv('../dataset/votes_senate_clausen/'+clausen_cat_dic[i]+'/congress_'+str(n_congress)+'.csv', index=False)

    for i in peltzman_cat_dic.keys():
        filtered_df = df_congress[df_congress['Peltzman'] == i]
        if not filtered_df.empty:
            filtered_df.to_csv('../dataset/votes_senate_peltzman/'+peltzman_cat_dic[i]+'/congress_'+str(n_congress)+'.csv', index=False)

100%|██████████| 41/41 [00:26<00:00,  1.57it/s]


_________________________________________________________________________

### Create edgelists

Create folders

In [40]:
os.makedirs('../dataset/edgelists/', exist_ok=True)

os.makedirs('../dataset/edgelists/votes_house_clausen_edges/', exist_ok=True)
os.makedirs('../dataset/edgelists/votes_house_peltzman_edges/', exist_ok=True)

os.makedirs('../dataset/edgelists/votes_senate_clausen_edges/', exist_ok=True)
os.makedirs('../dataset/edgelists/votes_senate_peltzman_edges/', exist_ok=True)

In [41]:
for clausen in clausen_cat_dic.values():
	os.makedirs('../dataset/edgelists/votes_house_clausen_edges/'+clausen, exist_ok=True)
	os.makedirs('../dataset/edgelists/votes_senate_clausen_edges/'+clausen, exist_ok=True)

for peltzman in peltzman_cat_dic.values():
	os.makedirs('../dataset/edgelists/votes_house_peltzman_edges/'+peltzman, exist_ok=True)
	os.makedirs('../dataset/edgelists/votes_senate_peltzman_edges/'+peltzman, exist_ok=True)

Create house edgelists

In [24]:
clausen_cat_dic

{'Agriculture': 'Agriculture',
 'Civil Liberties': 'Civil_Liberties',
 'Foreign and Defense Policy': 'Foreign_and_Defense_Policy',
 'Government Management': 'Government_Management',
 'Miscellaneous Policy': 'Miscellaneous_Policy',
 'Social Welfare': 'Social_Welfare'}

In [42]:
for clausen in clausen_cat_dic.values():
	for csv in glob.glob('../dataset/votes_house_clausen/'+clausen+'/*'):
		n_congress = int(os.path.basename(csv).replace('congress_', '').replace('.csv', ''))
		df_congress = pd.read_csv(csv)
		edgelist = create_edgelist_from_congress(df_congress, member_party_dict)
		edgelist.to_csv('../dataset/edgelists/votes_house_clausen_edges/'+clausen+'/congress_'+str(n_congress)+'.csv', index=False)

100%|██████████| 37/37 [00:02<00:00, 17.07it/s]


../dataset/votes_house_clausen/Foreign_and_Defense_Policy/congress_80.csv


100%|██████████| 27/27 [00:01<00:00, 17.99it/s]


../dataset/votes_house_clausen/Foreign_and_Defense_Policy/congress_90.csv


100%|██████████| 71/71 [00:04<00:00, 15.94it/s]


...

100%|██████████| 37/37 [00:02<00:00, 15.71it/s]


../dataset/votes_house_clausen/Foreign_and_Defense_Policy/congress_98.csv


100%|██████████| 190/190 [00:13<00:00, 13.91it/s]


../dataset/votes_house_clausen/Foreign_and_Defense_Policy/congress_105.csv


 63%|██████▎   | 156/248 [00:12<00:06, 13.29it/s]

In [ ]:
for peltzman in peltzman_cat_dic.values():
	for csv in glob.glob('../dataset/votes_house_peltzman/'+peltzman+'/*'):
		n_congress = int(os.path.basename(csv).replace('congress_', '').replace('.csv', ''))
		df_congress = pd.read_csv(csv)
		edgelist = create_edgelist_from_congress(df_congress, member_party_dict)
		edgelist.to_csv('../dataset/edgelists/votes_house_peltzman_edges/'+clausen+'/congress_'+str(n_congress)+'.csv', index=False)

Create senate edgelist

In [ ]:
for clausen in clausen_cat_dic.values():
	for csv in glob.glob('../dataset/votes_senate_clausen/'+clausen+'/*'):
		n_congress = int(os.path.basename(csv).replace('congress_', '').replace('.csv', ''))
		df_congress = pd.read_csv(csv)
		edgelist = create_edgelist_from_congress(df_congress, member_party_dict)
		edgelist.to_csv('../dataset/edgelists/votes_senate_clausen_edges/'+clausen+'/congress_'+str(n_congress)+'.csv', index=False)

In [ ]:
for peltzman in peltzman_cat_dic.values():
	for csv in glob.glob('../dataset/votes_senate_peltzman/'+peltzman+'/*'):
		n_congress = int(os.path.basename(csv).replace('congress_', '').replace('.csv', ''))
		df_congress = pd.read_csv(csv)
		edgelist = create_edgelist_from_congress(df_congress, member_party_dict)
		edgelist.to_csv('../dataset/edgelists/votes_senate_peltzman_edges/'+clausen+'/congress_'+str(n_congress)+'.csv', index=False)

_______________________________________________________________________________________

### Perform the analysis on the disaggregated data

In [21]:
def compute_kde_threshold(df, weight):
    
	def _midpoint(p1, p2):
		return {'x': (p1['x']+p2['x'])/2, 'y': (p1['y']+p2['y'])/2}

	def line_intersection(in_state, out_state, intersect_points):
		index_in = np.argmax(in_state[1])
		index_out = np.argmax(out_state[1])

        # points of the mean of the distributions 
		point_in={'x': in_state[0][index_in], 'y': in_state[1][index_in]}
		point_out={'x': out_state[0][index_out], 'y': out_state[1][index_out]}

        # medianpoint (mean of the means) of the two distributions
		midpoint = _midpoint(point_in, point_out)
        
        #find index of intersection closer to midpoint
		index_closer = np.argmin([np.sqrt( (p[0] - midpoint['x'])**2 + (p[1] - midpoint['y'])**2 ) for p in intersect_points])

        # return x value of closer intersection
		return intersect_points[index_closer][0]
    

	#label = "agree" if weight == 1 else "disagree"
	x0 = df.loc[(df['state']=='in')&(df['weight'] == weight)]['perc']
	x1 = df.loc[(df['state']=='out')&(df['weight'] == weight)]['perc']

	bw = len(x0)**(-1./(2+4))
	kde0 = gaussian_kde(x0, bw_method=bw)
	bw = len(x1)**(-1./(2+4))
	kde1 = gaussian_kde(x1, bw_method=bw)

	xmin = min(x0.min(), x1.min())
	xmax = max(x0.max(), x1.max())
	dx = 0.2 * (xmax - xmin) # add a 20% margin, as the kde is wider than the data
	xmin -= dx
	xmax += dx

	x = np.linspace(xmin, xmax, 500)
	kde0_x = kde0(x)
	kde1_x = kde1(x)

	idx = np.argwhere(np.diff(np.sign(kde0_x - kde1_x))).flatten()

	threshold = line_intersection([x, kde0_x], [x, kde0_x], [[x,y] for x,y in zip (x[idx], kde1_x[idx])])

	return threshold

In [24]:
import sys
sys.path.append('..')

from sygno.signed_networkx import draw_signed_networkx